In [2]:
import numpy as np
import torch
import torch.nn.functional as F
from io import BytesIO
import scipy.io as sio
import h5py
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd
from scipy.signal import butter, sosfiltfilt, cheby2, periodogram, correlate
from pyampd.ampd import find_peaks, find_peaks_adaptive
from scipy.signal import find_peaks as fp
import pickle
import matplotlib.pyplot as plt

In [2]:
dset = h5py.File('/root/data/PPG2BP/datasets/Part1234.mat', 'r') 
idxs = np.arange(0,12000)
delete_idxs = np.array([])
for i in idxs:
    if dset[str(i)].shape[1] < 6000:
        delete_idxs = np.append(delete_idxs, i)
idxs = np.delete(idxs, delete_idxs.astype(int))

In [4]:
def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return array[idx]

counter = 0
data = {}
data_idxs = {}
for subj in idxs:
    sig = dset[str(subj)]
    ppg_sig = sig[0,:]
    ecg_sig = sig[2,:]
    abp_sig = sig[1, :]
    seg_len = 1000
    subj_data = np.empty((1, 3, seg_len))
    subj_idxs = np.empty((1, 7))
    for i in range(seg_len-int(seg_len/2), len(ppg_sig)-int(seg_len/2), int(seg_len/2)):
        try:
            # correct for signal drift
            lag = seg_len-correlate(ppg_sig[i-int(seg_len/2):i+int(seg_len/2)], abp_sig[i-int(seg_len/2):i+int(seg_len/2)]).argmax()
            ppg_seg = ppg_sig[i-int(seg_len/2)-lag:i+int(seg_len/2)-lag]
            ecg_seg = ecg_sig[i-int(seg_len/2)-lag:i+int(seg_len/2)-lag]
            abp_seg = abp_sig[i-int(seg_len/2):i+int(seg_len/2)]

            ppg_max_idxs = find_peaks(ppg_seg)
            ecg_max_idxs = find_peaks(ecg_seg)
            sbp_idxs = find_peaks(abp_seg)
            dbp_idxs = find_peaks(-abp_seg)

            sbp_idxs = sbp_idxs[sbp_idxs > dbp_idxs[0]]
            nearest_dbp_idxs = np.array([])
            for j in range(len(sbp_idxs)):
                nearest_dbp_idxs = np.append(nearest_dbp_idxs, find_nearest(dbp_idxs[dbp_idxs < sbp_idxs[j]], sbp_idxs[j]))
            nearest_dbp_idxs = nearest_dbp_idxs.astype(int)

            nearest_ppgmax_idxs = np.array([])
            for j in range(len(sbp_idxs)):
                nearest_ppgmax_idxs = np.append(nearest_ppgmax_idxs, find_nearest(ppg_max_idxs, sbp_idxs[j]))
            nearest_ppgmax_idxs = nearest_ppgmax_idxs.astype(int) 

            # Physiological mask
            sys_t = ((sbp_idxs-nearest_dbp_idxs)/125) > 0.5
            pp = (abp_seg[sbp_idxs]-abp_seg[nearest_dbp_idxs]) > 70
            mx = abp_seg.max() > 200
            mn = abp_seg.min() < 30
            consec_sbp = np.absolute(np.diff(abp_seg[sbp_idxs])) > 40
            consec_dbp = np.absolute(np.diff(abp_seg[nearest_dbp_idxs])) > 40
            hr = (60/np.diff(sbp_idxs/125) > 240) | (60/np.diff(sbp_idxs/125) < 60)
            num_peaks_pe = (len(sbp_idxs) - len(ppg_max_idxs)) > 1
            num_peaks_pa = (len(ecg_max_idxs) - len(ppg_max_idxs)) > 1
            dt_sbp_ppg = np.absolute(nearest_ppgmax_idxs - sbp_idxs)/125 > 0.15
            mask = ~sys_t.all() & ~pp.all() & ~mx.all() & ~mn.all() & ~consec_sbp.all() & ~consec_dbp.all() & ~hr.all() & ~dt_sbp_ppg.all() & ~num_peaks_pe & ~num_peaks_pa
            
            ### SQI
            # relpower ecg
            freq, Pxx = periodogram(ecg_seg, fs=125, scaling='spectrum')
            relpower_ecg = (Pxx[(freq>=1)&(freq<=40)]/125).sum()/(Pxx[freq>40]/125).sum()
            # skewness ecg
            skew_ecg = np.mean(((ecg_seg-ecg_seg.mean())/ecg_seg.std())**3)
            # kurtosis ecg
            kurt_ecg = np.mean(((ecg_seg-ecg_seg.mean())/ecg_seg.std())**4)
            
            # goodness ppg
            freq, Pxx = periodogram(ppg_seg, fs=125, scaling='spectrum')
            Pxx_peaks = fp(Pxx, distance=int(len(freq)/50))[0]
            df = 0.5
            psd_sum = (Pxx[(freq>=0.5)&(freq<=5)]/125).sum()
            psd_in = (Pxx[((freq>freq[Pxx.argmax()]-df)&(freq<freq[Pxx.argmax()]+df))]/125).sum()
            goodness_ppg = psd_in/(psd_sum-psd_in)
            # perfusion index ppg
            pi_ppg = (ppg_seg.max() - ppg_seg.min())/np.absolute(ppg_seg.mean())
            # skewness ppg
            skew_ppg = np.mean(((ppg_seg-ppg_seg.mean())/ppg_seg.std())**3)
            # kurtosis ppg
            kurt_ppg = np.mean(((ppg_seg-ppg_seg.mean())/ppg_seg.std())**4)
            
            if mask:
                subj_data = np.concatenate((subj_data, np.expand_dims(np.concatenate((np.expand_dims(ppg_seg, axis=0), np.expand_dims(ecg_seg, axis=0), np.expand_dims(abp_seg, axis=0))), axis=0)), axis=0)
                subj_idxs = np.vstack((subj_idxs, np.array([relpower_ecg, skew_ecg, kurt_ecg, goodness_ppg, pi_ppg, skew_ppg, kurt_ppg]).reshape(1, -1)))
        except Exception as e:
            print(e)

    if subj_data.shape[0] > 1:
        subj_data = subj_data[1:, :, ::4]
        subj_idxs = subj_idxs[1:]
        data[str(subj)] = subj_data
        data_idxs[str(subj)] = subj_idxs
        print(subj, subj_data.shape)
#         fig, ax = plt.subplots(2,1, figsize=(15, 10))
#         ax[0].plot(ppg_seg)
#         ax[1].plot(abp_seg)

integer division or modulo by zero
0 (115, 3, 250)
integer division or modulo by zero
1 (116, 3, 250)
2 (96, 3, 250)
integer division or modulo by zero
3 (137, 3, 250)
integer division or modulo by zero
4 (131, 3, 250)
5 (2, 3, 250)
6 (68, 3, 250)
14 (44, 3, 250)
integer division or modulo by zero
15 (126, 3, 250)
16 (124, 3, 250)
integer division or modulo by zero
17 (37, 3, 250)
integer division or modulo by zero
18 (108, 3, 250)
24 (27, 3, 250)
integer division or modulo by zero
28 (6, 3, 250)
29 (4, 3, 250)
33 (8, 3, 250)
34 (10, 3, 250)
35 (133, 3, 250)
36 (139, 3, 250)
37 (94, 3, 250)
38 (30, 3, 250)
39 (118, 3, 250)
40 (142, 3, 250)
41 (144, 3, 250)
42 (20, 3, 250)
44 (10, 3, 250)
45 (12, 3, 250)
46 (134, 3, 250)
50 (20, 3, 250)
62 (20, 3, 250)
64 (5, 3, 250)
integer division or modulo by zero
66 (130, 3, 250)
integer division or modulo by zero
67 (145, 3, 250)
integer division or modulo by zero
70 (11, 3, 250)
integer division or modulo by zero
71 (14, 3, 250)
integer division 

zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction opera

380 (5, 3, 250)
integer division or modulo by zero
382 (15, 3, 250)
383 (17, 3, 250)
integer division or modulo by zero
385 (44, 3, 250)
387 (14, 3, 250)
388 (20, 3, 250)
integer division or modulo by zero
389 (19, 3, 250)
391 (108, 3, 250)
393 (51, 3, 250)
394 (16, 3, 250)
395 (32, 3, 250)
396 (13, 3, 250)
397 (26, 3, 250)
399 (47, 3, 250)
400 (28, 3, 250)
401 (102, 3, 250)
402 (108, 3, 250)
403 (123, 3, 250)
404 (56, 3, 250)
405 (119, 3, 250)
406 (115, 3, 250)
407 (41, 3, 250)
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
408 (99, 3, 250)
409 (137, 3, 250)
410 (55, 3, 250)
412 (30, 3, 250)
413 (62, 3, 250)
integer division or modulo by zero
414 (33, 3, 250)
integer division or modulo by zero
zero-size array to reduction operation minimum which has no identit

zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction opera

zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction opera

...

zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction opera

zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction opera

zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction opera

zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction operation minimum which has no identity
zero-size array to reduction opera

11870 (14, 3, 250)
integer division or modulo by zero
11871 (75, 3, 250)
11872 (74, 3, 250)
11873 (73, 3, 250)
11874 (37, 3, 250)
integer division or modulo by zero
11875 (36, 3, 250)
integer division or modulo by zero
11876 (73, 3, 250)
integer division or modulo by zero
11877 (26, 3, 250)
11878 (50, 3, 250)
11879 (43, 3, 250)
11880 (33, 3, 250)
integer division or modulo by zero
11881 (45, 3, 250)
integer division or modulo by zero
11884 (55, 3, 250)
11885 (89, 3, 250)
11886 (19, 3, 250)
integer division or modulo by zero
11890 (7, 3, 250)
11891 (48, 3, 250)
11894 (29, 3, 250)
integer division or modulo by zero
11895 (29, 3, 250)
11896 (73, 3, 250)
11900 (43, 3, 250)
integer division or modulo by zero
11901 (17, 3, 250)
11902 (19, 3, 250)
integer division or modulo by zero
11903 (46, 3, 250)
integer division or modulo by zero
11904 (61, 3, 250)
11905 (64, 3, 250)
integer division or modulo by zero
11906 (52, 3, 250)
integer division or modulo by zero
11907 (54, 3, 250)
11908 (23, 3, 

In [5]:
import pickle
with open('data1_all.pickle', 'wb') as f:
    pickle.dump(data, f)
with open('data1_all_idxs.pickle', 'wb') as f:
    pickle.dump(data_idxs, f)

In [24]:
with open('data1_all.pickle', 'rb') as f:
    data = pickle.load(f)
with open('data1_all_idxs.pickle', 'rb') as f:
    data_idxs = pickle.load(f)

In [25]:
data_idxs['0'].shape

(115, 7)

In [14]:
sqis = np.empty((1, 7))
for i in np.fromiter(data_idxs.keys(), dtype=int).astype(str):
    sqis = np.vstack((sqis, data_idxs[str(i)]))
    print(i)
sqis = sqis[1:, :]

0
1
2
3
4
5
6
14
15
16
17
18
24
28
29
33
34
35
36
37
38
39
40
41
42
44
45
46
50
62
64
66
67
70
71
73
76
78
79
80
81
82
83
85
86
87
88
89
93
94
95
98
99
100
101
102
103
104
105
106
108
109
110
111
114
115
116
117
118
120
121
122
124
125
126
127
128
129
130
131
132
133
134
139
140
141
143
144
145
146
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
323
324
325
326
327


2573
2575
2576
2577
2578
2579
2584
2587
2588
2589
2590
2591
2592
2594
2595
2597
2598
2599
2600
2620
2622
2624
2626
2629
2630
2631
2634
2635
2636
2637
2638
2639
2640
2647
2648
2649
2650
2651
2652
2653
2654
2655
2657
2658
2660
2661
2662
2663
2664
2667
2668
2669
2671
2672
2675
2676
2677
2680
2681
2682
2685
2686
2687
2691
2696
2699
2700
2701
2702
2703
2705
2708
2712
2713
2715
2720
2724
2726
2727
2728
2729
2732
2733
2734
2735
2736
2739
2740
2743
2745
2746
2747
2748
2749
2751
2752
2753
2754
2755
2757
2758
2759
2762
2763
2764
2766
2767
2769
2770
2771
2772
2773
2774
2776
2779
2780
2781
2783
2784
2785
2786
2787
2788
2789
2793
2794
2800
2801
2803
2805
2806
2807
2808
2811
2812
2817
2818
2819
2822
2824
2830
2837
2858
2860
2861
2863
2869
2899
2900
2902
2903
2904
2907
2909
2913
2914
2922
2923
2928
2930
2932
2933
2934
2938
2939
2940
2941
2942
2944
2945
2947
2949
2951
2952
2960
2964
2967
2968
2970
2971
2974
2975
2976
2977
2978
2979
2980
2981
2985
2987
2988
2989
2990
2991
2992
2993
2994
2995
2996
2997


5485
5489
5490
5492
5493
5494
5495
5496
5497
5498
5500
5501
5503
5504
5505
5506
5508
5509
5510
5511
5513
5514
5515
5516
5517
5518
5519
5520
5521
5522
5523
5524
5525
5526
5527
5528
5529
5530
5531
5532
5534
5536
5537
5538
5539
5540
5541
5542
5543
5544
5545
5547
5550
5552
5553
5554
5555
5556
5557
5558
5559
5560
5561
5562
5563
5564
5565
5566
5567
5568
5569
5570
5571
5572
5573
5574
5575
5576
5577
5578
5579
5580
5581
5582
5583
5584
5586
5587
5588
5590
5591
5592
5593
5594
5595
5596
5597
5598
5599
5600
5601
5602
5604
5605
5606
5607
5608
5609
5610
5611
5612
5613
5614
5615
5616
5617
5618
5619
5620
5621
5622
5623
5624
5625
5626
5627
5628
5629
5631
5632
5633
5634
5635
5636
5637
5638
5639
5644
5645
5649
5650
5651
5652
5653
5654
5655
5657
5658
5660
5661
5662
5665
5666
5668
5672
5675
5676
5677
5678
5680
5681
5682
5683
5684
5685
5688
5689
5690
5691
5692
5693
5694
5695
5696
5697
5698
5699
5700
5701
5702
5703
5704
5705
5706
5707
5708
5711
5712
5713
5714
5715
5716
5717
5718
5719
5720
5721
5722
5723
5724


7573
7574
7575
7576
7578
7579
7581
7582
7583
7584
7586
7587
7588
7589
7591
7592
7593
7594
7595
7597
7599
7600
7601
7602
7603
7604
7606
7607
7608
7609
7610
7611
7613
7615
7617
7618
7619
7621
7622
7623
7625
7626
7627
7628
7629
7630
7631
7632
7633
7634
7635
7636
7639
7640
7641
7642
7643
7644
7645
7646
7647
7648
7649
7650
7651
7652
7653
7654
7655
7656
7657
7658
7659
7660
7662
7663
7664
7665
7666
7667
7668
7669
7670
7671
7672
7673
7674
7675
7676
7677
7678
7679
7680
7681
7682
7683
7684
7685
7686
7688
7690
7691
7693
7696
7697
7698
7699
7700
7701
7702
7703
7705
7706
7707
7708
7709
7710
7711
7712
7713
7714
7715
7716
7717
7718
7719
7720
7721
7723
7725
7727
7728
7729
7730
7731
7732
7733
7734
7736
7737
7739
7740
7741
7742
7744
7745
7747
7748
7750
7752
7754
7755
7756
7757
7758
7759
7760
7761
7762
7764
7765
7766
7767
7769
7770
7771
7773
7774
7775
7777
7778
7779
7780
7781
7782
7783
7784
7785
7786
7787
7788
7789
7792
7793
7794
7795
7796
7797
7799
7801
7802
7803
7804
7805
7807
7808
7809
7811
7813
7814


9828
9842
9843
9844
9847
9849
9851
9852
9853
9854
9855
9856
9859
9861
9862
9863
9864
9865
9866
9867
9868
9869
9871
9872
9873
9874
9875
9876
9877
9878
9879
9880
9881
9882
9883
9884
9886
9887
9888
9889
9890
9891
9892
9893
9894
9895
9896
9897
9899
9900
9901
9902
9903
9904
9905
9906
9907
9908
9910
9911
9912
9913
9914
9915
9916
9917
9918
9919
9920
9921
9922
9923
9924
9925
9926
9927
9933
9934
9935
9936
9937
9938
9939
9940
9941
9942
9943
9944
9945
9946
9947
9948
9949
9950
9951
9952
9953
9954
9955
9956
9957
9958
9959
9960
9965
9967
9968
9973
9974
9975
9977
9978
9979
9980
9982
9983
9984
9985
9986
9988
9989
9991
9992
9993
9994
9995
9996
9998
9999
10000
10001
10002
10003
10004
10006
10007
10008
10009
10010
10011
10012
10013
10014
10015
10016
10017
10018
10020
10021
10022
10023
10026
10027
10028
10029
10030
10031
10032
10033
10034
10035
10036
10037
10040
10041
10042
10043
10044
10045
10046
10047
10048
10050
10051
10052
10053
10054
10056
10057
10059
10062
10063
10064
10065
10066
10067
10068
10069
1

11843
11844
11847
11848
11849
11850
11851
11852
11853
11854
11855
11856
11857
11858
11859
11860
11861
11862
11863
11866
11867
11868
11869
11870
11871
11872
11873
11874
11875
11876
11877
11878
11879
11880
11881
11884
11885
11886
11890
11891
11894
11895
11896
11900
11901
11902
11903
11904
11905
11906
11907
11908
11909
11910
11911
11912
11913
11914
11915
11919
11921
11923
11924
11926
11927
11929
11930
11931
11933
11934
11936
11937
11939
11940
11941
11942
11944
11945
11946
11949
11950
11951
11954
11957
11958
11959
11960
11964
11965
11966
11967
11968
11969
11970
11971
11972
11973
11974
11976
11977
11978
11979
11980
11981
11982
11983
11986
11987
11992
11993
11994
11995
11996
11998


In [15]:
sqis.shape

(414860, 7)

In [17]:
with open('data1_all_dist.pickle', 'wb') as f:
    pickle.dump(sqis, f)

In [19]:
with open('data1_all_dist.pickle', 'rb') as f:
    data_idxs = pickle.load(f)

In [52]:
lbs = [0, 25, 45, 50, 75]
ubs = [25, 50, 55, 75, 100]
sqi_names = ['relpower_ecg', 'skew_ecg', 'kurt_ecg', 'goodness_ppg', 'pi_ppg', 'skew_ppg', 'kurt_ppg']

for sqi_idx in range(len(sqi_names)):
    print(sqi_names[sqi_idx])
    for bound_idx in range(len(lbs)):
        new_data = {}
        lb_val = np.percentile(sqis[:, sqi_idx], lbs[bound_idx])
        ub_val = np.percentile(sqis[:, sqi_idx], ubs[bound_idx])
        for i in np.fromiter(data.keys(), dtype=int).astype(str):
            d = data_idxs[str(i)][:, sqi_idx]
            d_mask = (d > lb_val) & (d < ub_val)
            if d_mask.any():
                new_data[str(i)] = data[str(i)][np.nonzero(d_mask)[0], :, :]
        print(len(new_data), lb_val, ub_val)
        with open('/root/data/PPG2BP/datasets/sqi_data/data1_' + sqi_names[sqi_idx] + '_' + str(int(lbs[bound_idx])) + '_' + str(int(ubs[bound_idx])) + '.pickle', 'wb') as f:
            pickle.dump(new_data, f)

relpower_ecg
3842 0.7332679988163117 125.63432260890393
5429 125.63432260890393 313.8967101125828
3911 260.39828395021743 378.8312934104202
4719 313.8967101125828 888.7312539881275
2910 888.7312539881275 83537.1695123104
skew_ecg
2903 -6.52953716729391 0.8895953211283899
3762 0.8895953211283899 2.7338978295554877
2225 2.467911272825618 2.9071267987527123
3222 2.7338978295554877 3.444494613545935
2788 3.444494613545935 10.370985069315442
kurt_ecg
3416 1.444012669034782 8.605359631597588
3833 8.605359631597588 12.969123157136536
2570 12.166163134713472 13.763036475546755
3740 12.969123157136536 16.44385114892978
3205 16.44385114892978 159.39488924255426
goodness_ppg
5399 -1431.2179011830206 2.0066720239041445
6100 2.0066720239041445 2.919891016558176
4810 2.7315170911712876 3.122289589340949
5628 2.919891016558176 4.112107970729795
4203 4.112107970729795 1229.3356665093977
pi_ppg
5703 0.6245560753386072 1.1694917584579496
6701 1.1694917584579496 1.2390912470781956
5746 1.2258081996743404

In [53]:
!ls /root/data/PPG2BP/datasets/sqi_data/

data1_goodness_ppg_0_25.pickle	  data1_pi_ppg_50_75.pickle
data1_goodness_ppg_25_50.pickle   data1_pi_ppg_75_100.pickle
data1_goodness_ppg_45_55.pickle   data1_relpower_ecg_0_25.pickle
data1_goodness_ppg_50_75.pickle   data1_relpower_ecg_25_50.pickle
data1_goodness_ppg_75_100.pickle  data1_relpower_ecg_45_55.pickle
data1_kurt_ecg_0_25.pickle	  data1_relpower_ecg_50_75.pickle
data1_kurt_ecg_25_50.pickle	  data1_relpower_ecg_75_100.pickle
data1_kurt_ecg_45_55.pickle	  data1_skew_ecg_0_25.pickle
data1_kurt_ecg_50_75.pickle	  data1_skew_ecg_25_50.pickle
data1_kurt_ecg_75_100.pickle	  data1_skew_ecg_45_55.pickle
data1_kurt_ppg_0_25.pickle	  data1_skew_ecg_50_75.pickle
data1_kurt_ppg_25_50.pickle	  data1_skew_ecg_75_100.pickle
data1_kurt_ppg_45_55.pickle	  data1_skew_ppg_0_25.pickle
data1_kurt_ppg_50_75.pickle	  data1_skew_ppg_25_50.pickle
data1_kurt_ppg_75_100.pickle	  data1_skew_ppg_45_55.pickle
data1_pi_ppg_0_25.pickle	  data1_skew_ppg_50_75.pickle
data1_pi_ppg_25_50.pickle	  data1_skew_p